In [ ]:
# import sys; sys.path

In [ ]:
from pyroclastmpm import (
    NoSlipWall,
    LinearElastic,
    NewtonFluid,
    ParticlesContainer,
    NodesContainer,
    USL,
    MUSL,
    APIC,
    LinearShapeFunction,
    set_globals,
)

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def create_circle(
    center: np.array, radius: float, cell_size: float, ppc: int = 1
):
    """Create a 2D circle

    :param center: center of circle
    :param radius: radius of circle
    :param cell_size: cell size of background grid
    :param ppc: particles per cell, defaults to 1
    """
    start = center - radius
    end = center + radius
    spacing = cell_size / ppc
    tol = +0.002  # prevents points

    x = np.arange(start[0], end[0] + spacing, spacing)
    y = np.arange(start[1], end[1] + spacing, spacing)
    z = np.zeros(len(x))
    xv, yv, zv = np.meshgrid(x, y, z)
    grid_coords = np.array(
        list(zip(xv.flatten(), yv.flatten(), zv.flatten()))
    ).astype(np.float64)

    circle_mask = (grid_coords[:, 0] - center[0]) ** 2 + (
        grid_coords[:, 1] - center[1]
    ) ** 2 < radius**2 + tol

    return grid_coords[circle_mask]

In [ ]:
domain_start = np.array([0.0, 0.0, 0.0])
domain_end = np.array([1.0, 1.0, 1.0])
cell_size = 1 / 20
ppc = 4

rho0 = 1000

# Define global simulation parameters
set_globals(
    dimension=2,
    dt=0.001,
    shape_function=LinearShapeFunction,
    output_directory="./output",
)

In [ ]:
# setup grid

nodes = NodesContainer(
    node_start=domain_start, node_end=domain_end, node_spacing=cell_size
)

print(f"Total number of cells: {nodes.num_nodes_total}")

In [ ]:
# setup particles

circle1 = create_circle(
    center=np.array([0.3, 0.3, 0.0]), radius=0.15, cell_size=cell_size, ppc=ppc
)

circle2 = create_circle(
    center=np.array([0.7, 0.7, 0.0]), radius=0.15, cell_size=cell_size, ppc=ppc
)


plt.scatter(circle1[:, 0], circle1[:, 1])
plt.scatter(circle2[:, 0], circle2[:, 1])

plt.xlim((0, 1))
plt.ylim((0, 1))

positions = np.vstack((circle1, circle2))

velocities1 = np.ones(circle1.shape) * 0.1
velocities2 = np.ones(circle2.shape) * -0.1
velocities = np.vstack((velocities1, velocities2))
velocities[:, 2] = 0.0

color1 = np.zeros(len(circle1))
color2 = np.ones(len(circle2))
colors = np.concatenate([color1, color2]).astype(int)


masses = np.ones(len(positions)) * rho0 * cell_size * cell_size
volumes = np.ones(len(positions)) * cell_size * cell_size

particles = ParticlesContainer(
    positions=positions,
    masses=masses,
    volumes=volumes,
    velocities=velocities,
    colors=colors,
)


material = LinearElastic(E=1000, pois=0.3)

particles.materials = [material, material]  # two color

print(f"Total number of particles {particles.num_particles}")

In [ ]:
MPM = USL(
    total_steps=2600,  # 3 seconds
    output_steps=400,
    output_start=0,
    particles=particles,
    nodes=nodes,
)
MPM.run()

In [ ]:
import glob
import pandas as pd
import numpy as np

all_files = glob.glob("output/particles*.csv")

df_particles_list = []

for i, file in enumerate(all_files):
    time = i * 200 * 0.001
    df = pd.read_csv(file)
    df["time"] = time
    df_particles_list.append(df)

In [ ]:
df_full = pd.concat(df_particles_list)

vels = df_full[["Velocity:0", "Velocity:1", "Velocity:2"]].values
masses = df_full["Mass"].values

df_full["KE"] = 0.5 * np.einsum("ij,ij->i", vels, vels) * masses

grper = df_full.groupby(["time"]).sum().reset_index()

grper.plot(x="time", y="KE")